**Using this template**

1. Change notebook title to assignment title you working on, eg. `Graded Challenge 1 or Milestone 1`
2. Put your identity such as name and batch number below notebook title.
3. Write your description before do any work. State your purpose of this assignment and what you are trying to achieve. 
4. Look at the rubric of this assignment and make sure you understand the criteria.
5. Code your solution in the cell provided below the working area section.
6. State your conlusions, findings, and any other relevant information in the cell provided below the conclusions sections.
7. Save this notebook and rename it to assignment title you working on  eg. `Graded Challenge 1 or Milestone 1`
8. Push your assignment before deadline.
9. Good Luck!

# Graded Challenge 2

1. Name: Jason Rich Darmawan Onggo Putra
2. Batch: 016 RMT

## Description

### Objective

Analysis of the waste management and disposal system in the city of Austin. The analysis result will be the condition of the waste management and disposal system. In addition, the prediction of the weight of waste in the city of Austin on July 10, 2021.

### Problem Statement

#### Problems 1

- [ ] What is the average **time between delivery** of every dropoff site? 
= [ ] What is the **maximum** waste weight collected of every dropoff site **per day**?
- [ ] What is the **average** waste weight collected of every dropoff site **per delivery**?
- [ ] What is the **median** waste weight collected of every dropoff site **per delivery**?
- [ ] What is the **mode** waste weight of every dropoff site **per delivery**?

### Problems 2

- [ ] What is the **maximum** waste weight collected of every dropoff site in July 10, 2021?

## Working Area

```txt
Put your code, analysis, everything below this line

Make sure to check everything the rubric requires before you submit
```

## Data Retrieve

1. project: `bigquery-public-data`
2. dataset: [austin_waste](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m4!1m3!3m2!1sbigquery-public-data!2saustin_waste)
3. table: [waste_and_diversion](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m5!1m4!4m3!1sbigquery-public-data!2saustin_waste!3swaste_and_diversion)

**The query**:
```
SELECT report_date, load_time, load_type, load_weight, dropoff_site 
FROM `bigquery-public-data.austin_waste.waste_and_diversion`
WHERE report_date BETWEEN '2021-01-01' AND '2021-07-09';
```

## Import Library

In [87]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from enum import Enum

## Data Preprocessing

Issues:
- [ ] column `report_date` dtype is `object`.
- [ ] column `load_time` dtype is `object`.
- [ ] column `load_weight` have `NaN` rows.

In [88]:
df = pd.read_csv("./h8dsft_P0GC2_jason_rich_darmawan_onggo_putra.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26157 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   26157 non-null  object 
 1   load_time     26157 non-null  object 
 2   load_type     26157 non-null  object 
 3   load_weight   25109 non-null  float64
 4   dropoff_site  26157 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1021.9+ KB


In [89]:
df.head()

,report_date,load_time,load_type,load_weight,dropoff_site
0,2021-07-08,2021-07-08 12:00:00 UTC,TIRES,2260.0,MRF
1,2021-07-08,2021-07-08 11:00:00 UTC,TIRES,1260.0,MRF
2,2021-07-08,2021-07-08 03:00:00 UTC,TIRES,800.0,MRF
3,2021-05-01,2021-05-01 12:29:00 UTC,TIRES,760.0,MRF
4,2021-07-03,2021-07-03 12:09:00 UTC,TIRES,1400.0,MRF


In [90]:
df.tail()

,report_date,load_time,load_type,load_weight,dropoff_site
26152,2021-03-01,2021-03-01 11:26:00 UTC,ORGANICS,12100.0,ORGANICS BY GOSH
26153,2021-03-01,2021-03-01 04:27:00 UTC,ORGANICS,10900.0,ORGANICS BY GOSH
26154,2021-03-01,2021-03-01 05:55:00 UTC,ORGANICS,21800.0,ORGANICS BY GOSH
26155,2021-03-01,2021-03-02 01:10:00 UTC,ORGANICS,5640.0,ORGANICS BY GOSH
26156,2021-03-01,2021-03-01 10:22:00 UTC,RECYCLED METAL,360.0,AUSTIN IRON AND METAL


### find dropoff_site that have never filled the `load_weight` column this year

dropoff_site that have never filled the `load_weight` column this year:
- [x] `ONION CREEK`
- [x] `PARK CENTER`
- [x] `ST. EDWARDS`
- [x] `WILCAB`

The decision is to drop these rows because the analyst does not have previous data (of each dropoff_site) to fill the missing value.

In [91]:
df.loc[:,['dropoff_site','load_weight']].groupby(['dropoff_site']).count()

,load_weight
dropoff_site,
AUSTIN IRON AND METAL,50
AUSTIN WOOD RECYCLING,2
BALCONES RECYCLING,3837
BALCONES WOODS,1
GREAT NORTHERN,6
HORNSBY BEND,1652
MRF,168
ON ROUTE HIGH DUMP,7
ONION CREEK,0


In [92]:
for dropoff_site in ['ONION CREEK', 'PARK CENTER', 'ST. EDWARDS', 'WILCAB']:
    print("{0} have at least once filled the `load_weight` column: {1}"
            .format(dropoff_site, df.loc[df['dropoff_site'] == dropoff_site, 'load_weight'].notna().any()))

ONION CREEK have at least once filled the `load_weight` column: False
PARK CENTER have at least once filled the `load_weight` column: False
ST. EDWARDS have at least once filled the `load_weight` column: False
WILCAB have at least once filled the `load_weight` column: False


In [93]:
df2 = df.loc[-df['dropoff_site'].str.contains('ONION CREEK|PARK CENTER|ST. EDWARDS|WILCAB')].copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25267 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   25267 non-null  object 
 1   load_time     25267 non-null  object 
 2   load_type     25267 non-null  object 
 3   load_weight   25109 non-null  float64
 4   dropoff_site  25267 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1.2+ MB


### find the dropoff_site that have NaN rows.

- [ ] GREAT NORTHERN have 64 NaN rows
- [ ] HORNSBY BEND have 3 NaN rows
- [ ] ON ROUTE HIGH DUMP have 74 NaN rows
- [ ] ORGANICS BY GOSH have 14 NaN rows
- [ ] TDS LANDFILL have 3 NaN rows

In [94]:
df2.loc[:,'load_weight'].isnull().groupby(df2['dropoff_site']).sum()

dropoff_site
AUSTIN IRON AND METAL     0
AUSTIN WOOD RECYCLING     0
BALCONES RECYCLING        0
BALCONES WOODS            0
GREAT NORTHERN           64
HORNSBY BEND              3
MRF                       0
ON ROUTE HIGH DUMP       74
ORGANICS BY GOSH         14
PARK BEND                 0
STEINER LANDFILL          0
TDS - MRF                 0
TDS LANDFILL              3
WESTFIELD                 0
Name: load_weight, dtype: int64

In [95]:
df3 = df2.copy()

#### fix: dropoff_site TDS LANDFILL

In [96]:
df3_DROPOFF_SITE_TDS_LANDFILL = df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL')].copy()
df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_weight'].isna())]

,report_date,load_time,load_type,load_weight,dropoff_site
15818,2021-06-11,2021-06-11 08:45:00 UTC,SWEEPING,NaN,TDS LANDFILL
22528,2021-01-27,2021-01-27 01:45:00 UTC,SWEEPING,NaN,TDS LANDFILL
24540,2021-02-12,2021-02-12 01:07:00 UTC,GARBAGE COLLECTIONS,NaN,TDS LANDFILL


##### fix: dropoff_site TDS LANDFILL; load_type SWEEPING

The skew value for `load_type` == `SWEEPING` is 0.18. The conclusion is the data distribution is normal. Therefore, the decision is:
1. Find the outliers with `mean +- 3 * std`. Then, take a decision to remove or keep the outliers.
2. Then, fill the NaN rows with the mean value.

In [97]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING = df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_type'] == 'SWEEPING')]
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].skew()

0.18290906329280904

In [98]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_MEAN = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].std()
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_STD = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].std()

df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_UPPER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_MEAN + 3 * df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_STD
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_LOWER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_MEAN - 3 * df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_STD

df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_LOWER_LIMIT, df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_UPPER_LIMIT

(-9097.925130423664, 18195.850260847328)

There are some outliers found. We will decide whether those outliers are natural variations or a mistake.

In [99]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[(df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_LOWER_LIMIT) 
                                           | (df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING_UPPER_LIMIT)]

,report_date,load_time,load_type,load_weight,dropoff_site
1225,2021-03-02,2021-03-02 05:18:00 UTC,SWEEPING,18820.0,TDS LANDFILL
11020,2021-05-06,2021-05-06 03:14:00 UTC,SWEEPING,21300.0,TDS LANDFILL
11021,2021-05-06,2021-05-06 08:52:00 UTC,SWEEPING,20620.0,TDS LANDFILL
11419,2021-05-10,2021-05-10 11:56:00 UTC,SWEEPING,19180.0,TDS LANDFILL
12600,2021-05-18,2021-05-18 01:49:00 UTC,SWEEPING,22760.0,TDS LANDFILL
15412,2021-06-09,2021-06-09 12:52:00 UTC,SWEEPING,18260.0,TDS LANDFILL
18897,2021-07-06,2021-07-06 01:43:00 UTC,SWEEPING,19880.0,TDS LANDFILL
19026,2021-07-07,2021-07-07 12:29:00 UTC,SWEEPING,18800.0,TDS LANDFILL
25517,2021-02-25,2021-02-25 01:33:00 UTC,SWEEPING,19760.0,TDS LANDFILL


We conclude that the outliers are natural variations. So, we will not remove the outliers.

In [100]:
df3_DROPOFF_SITE_TDS_LANDFILL_SWEEPING.loc[:,'load_weight'].value_counts().head()

11260.0    4
8440.0     3
7380.0     3
14900.0    3
8280.0     3
Name: load_weight, dtype: int64

In [101]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'SWEEPING') & (df3['load_weight'].isna()), 'load_weight'] = \
    df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_type'] == 'SWEEPING'), 'load_weight'].mean()

In [102]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_weight'].isna())]

,report_date,load_time,load_type,load_weight,dropoff_site
24540,2021-02-12,2021-02-12 01:07:00 UTC,GARBAGE COLLECTIONS,NaN,TDS LANDFILL


##### fix: dropoff_site TDS LANDFILL; load_type GARBAGE COLLECTIONS

The skew value for `load_type` == `GARBAGE COLLECTIONS` is 5.56. The conclusion is the data distribution is skewed positively. Therefore, the decision are:

1. Handle the outlier first with Inter-Quartile Range. Then, make the decision to remove or keep the outlierrs.
2. If the skew is STILL above 0,5 or below -0,5. We will fill the NaN rows with the median value.

In [103]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS = df3_DROPOFF_SITE_TDS_LANDFILL.loc[(df3_DROPOFF_SITE_TDS_LANDFILL['load_type'] == 'GARBAGE COLLECTIONS')]
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[:,'load_weight'].skew()

5.564210731462809

In [104]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q3, df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q1 = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[:,'load_weight'].quantile([.75,.25])
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_IQR = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q3 - df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q1
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q3 + df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_IQR * 1.5
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT = df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_Q1 - df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_IQR * 1.5

df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT, df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT

(-4495.0, 39625.0)

There are some outliers found. We will decide whether those outliers are natural variations or not.

In [105]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[(df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) 
                                                      | (df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT)]
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS') & (df3['load_weight'] == 145160)]
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS')].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8204 entries, 565 to 26109
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   8204 non-null   object 
 1   load_time     8204 non-null   object 
 2   load_type     8204 non-null   object 
 3   load_weight   8203 non-null   float64
 4   dropoff_site  8204 non-null   object 
dtypes: float64(1), object(4)
memory usage: 384.6+ KB


We conclude that the outliers are NOT natural variations. So, we will remove the outliers.

In [106]:
df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[:,['load_weight']].value_counts().head()

load_weight
21780.0        18
25900.0        17
19480.0        16
19120.0        16
24600.0        15
dtype: int64

In [107]:
"""
New knowledge: drop with `DataFrame.drop()`, do not try to replace the value, otherwise you will be left with all NaN rows.
"""
# df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS')] = \
#     df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS.loc[(df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) 
#                                                           & (df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT)
#                                                           | (df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS['load_weight'].isna())]
df3 = df3.drop(df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL')
                        & (df3['load_type'] == 'GARBAGE COLLECTIONS')
                        & ( (df3['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) | (df3['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT) )
                        & (df3['load_weight'].notna())]
                  .index)

df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS')] \
   .loc[(df3['load_weight'] < df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_LOWER_LIMIT) 
        | (df3['load_weight'] > df3_DROPOFF_SITE_TDS_LANDFILL_GARBAGE_COLLECTIONS_UPPER_LIMIT)]

,report_date,load_time,load_type,load_weight,dropoff_site


After removing the outliers, the skew changed to **-0.19**. We canconclude the data distribution is NOW normal. Therefore, we will replace the NaN rows with the mean value.

In [108]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS'), 'load_weight'].skew()

-0.19170335304750802

In [109]:
df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS') & (df3['load_weight'].isna()), 'load_weight'] = \
    df3.loc[(df3['dropoff_site'] == 'TDS LANDFILL') & (df3['load_type'] == 'GARBAGE COLLECTIONS'), 'load_weight'].mean()

### We will automate the process **Handle Outliers**

It took way too long to preprocess one dropoff_site. Therefore, we will streamline the process.

pseudo code:
```
def getSkew(dropoff_site: str, load_type: str) -> float: ...

def getMedian(dropoff_site: str, load_type: str) -> float: ...

class STRATEGY(Enum):
    STANDARD_DEVIATION = 0
    IQR = 1

def getUpperLowerLimit(dropoff_site: str, load_type: str, strategy: STRATEGY) -> tuple[float, float]: ...

for dropoff_site in dropoff_sites:
    for load_type in load_types[dropoff_site]:
        load_type_skew = getSkew(dropoff_site, load_type)
        if (load_type_skew < -0.5) | (load_type_skew > 0.5):
            lower_limit, upper_limit = getUpperLowerLimit(dropoff_site, load_type, strategy=STRATEGY.IQR)
            df3 = df3.drop(df3[(df3['dropoff_site'] == dropoff_site) 
                               & (df3['load_type'] == load_type) 
                               & ( (df3['load_weight'] < lower_limit ) | ( df3['load_weight'] > upper_limit ) )
                              ].index)
        else:
            print("dropoff_site {0} load_type {1} have normal data distribution".format(dropoff_site, load_type))
            pass
```

There are some dropoff_site and load_type that still have skewed data distribution after the outliers treament:

dropoff_site:
- **MRF**, with load type:
  - [ ] **TIRES**
- **TDS LANDFILL**, with load type:
  - [ ] **DEAD ANIMAL**
  - [ ] **MIXED LITTER**
  - [ ] **TIRES**
  - [ ] **RECYCLING - SINGLE STREAM**
- **ORGANICS BY GOSH**, with load type:
  - [ ] **DEAD ANIMAL**
- **ON ROUTE HIGH DUMP**, with load type:
  - [ ] **ORGANICS**
- **PARK BEND**, with load type:
  - [ ] **ORGANICS**

TODO: ask Instructor on how to handle skewed data distribution after the first outliers treatment failed.

In [110]:
class DEBUG_STRATEGY(Enum):
    INFO = 0
    VERBOSE = 1

DEBUG = DEBUG_STRATEGY.INFO

df4 = df3.copy()

def getSkew(dropoff_site: str, load_type: str) -> np.float64:
    return df4.loc[(df4['dropoff_site'] == dropoff_site)
                   & (df4['load_type'] == load_type), 'load_weight'] \
              .skew()

class OUTLIER_STRATEGY(Enum):
    IQR = 0

def getLowerUpperLimit(dropoff_site: str, load_type: str, strategy: OUTLIER_STRATEGY) -> tuple[float, float]:
    if strategy == OUTLIER_STRATEGY.IQR:
        Q1, Q3 = df4.loc[(df4['dropoff_site'] == dropoff_site)
                         & (df4['load_type'] == load_type), 'load_weight'] \
                    .quantile([.25, .75])
        IQR = Q3 - Q1
        return Q1 - IQR * 1.5, Q3 + IQR * 1.5
    else:
        raise NotImplementedError("currently just handle skewed data distribution")

getSkew('TDS LANDFILL', 'GARBAGE COLLECTIONS')
getLowerUpperLimit('TDS LANDFILL', 'GARBAGE COLLECTIONS', OUTLIER_STRATEGY.IQR)

for dropoff_site in df4.loc[:,'dropoff_site'].unique():
    for load_type in df4.loc[df4['dropoff_site'] == dropoff_site, 'load_type'].unique():
        dropoff_site_load_type_skew = getSkew(dropoff_site=dropoff_site, load_type=load_type)
        if np.isnan(dropoff_site_load_type_skew):
            """
            load_type_skew with nan value can happen because: there is not enough data, like only 2 rows.
            """
            if DEBUG == DEBUG_STRATEGY.VERBOSE:
                print("dropping dropoff_site {0} load_type {1} because have nan skew value".format(dropoff_site, load_type))
            df4 = df4.drop(df4.loc[(df4['dropoff_site'] == dropoff_site)
                                   & (df4['load_type'] == load_type)]
                              .index)
        elif (dropoff_site_load_type_skew < -0.5) | (dropoff_site_load_type_skew > 0.5):
            lower_limit, upper_limit = getLowerUpperLimit(dropoff_site=dropoff_site, load_type=load_type, strategy=OUTLIER_STRATEGY.IQR)
            df4 = df4.drop(df4.loc[(df4['dropoff_site'] == dropoff_site)
                                   & (df4['load_type'] == load_type)
                                   & ( (df4['load_weight'] < lower_limit) | (df4['load_weight'] > upper_limit) )]
                              .index)
            new_dropoff_site_load_type_skew = df4.loc[(df4['dropoff_site'] == dropoff_site) & (df4['load_type'] == load_type), 'load_weight'].skew()
            if DEBUG == DEBUG_STRATEGY.VERBOSE:
                print("dropping outliers in dropoff_site {0} load_type {1} because have skewed value: {2:.2f} -> {3:.2f}".format(dropoff_site, load_type, dropoff_site_load_type_skew, new_dropoff_site_load_type_skew))
            if (DEBUG == DEBUG_STRATEGY.INFO) & ((new_dropoff_site_load_type_skew < -0.5) | (new_dropoff_site_load_type_skew > 0.5)):
                print("after dropping outliers in dropoff_site {0} load_type {1} still have skewed value: {2:.2f} -> {3:.2f}".format(dropoff_site, load_type, dropoff_site_load_type_skew, new_dropoff_site_load_type_skew))
        else:
            if DEBUG == DEBUG_STRATEGY.VERBOSE:
                print("dropoff_site {0} load_type {1} have skew value {2:.2f}. The decision is not to drop the outliers".format(dropoff_site, load_type, dropoff_site_load_type_skew))

after dropping outliers in dropoff_site MRF load_type TIRES still have skewed value: 5.68 -> 0.56
after dropping outliers in dropoff_site TDS LANDFILL load_type DEAD ANIMAL still have skewed value: 2.25 -> 0.99
after dropping outliers in dropoff_site TDS LANDFILL load_type MIXED LITTER still have skewed value: 1.14 -> 0.63
after dropping outliers in dropoff_site TDS LANDFILL load_type TIRES still have skewed value: 3.09 -> -0.92
after dropping outliers in dropoff_site TDS LANDFILL load_type RECYCLING - SINGLE STREAM still have skewed value: 0.99 -> 0.99
after dropping outliers in dropoff_site ORGANICS BY GOSH load_type DEAD ANIMAL still have skewed value: 3.49 -> 1.47
after dropping outliers in dropoff_site ON ROUTE HIGH DUMP load_type ORGANICS still have skewed value: -0.88 -> -0.88
after dropping outliers in dropoff_site PARK BEND load_type ORGANICS still have skewed value: 0.54 -> 0.54


### Handle NaN rows.

Assuming we have done the outliers treatment.

~~pseudo code:~~ we only identify 2 combinations, not worth to automate ┗(｀O ´)┛
```
for dropoff_site in dropoff_sites:
    for load_type in dropoff_site['load_type']:
        dropoff_site_load_type_skew = getSkew(dropoff_site, load_type)
        if (dropoff_site_load_type_skew < -0.5) | (dropoff_site_load_type_skew > 0.5):
            # replace NaN with median
        elif:
            # repplace NaN with mean
```

We identified 2 combinations that have NaN rows and have normal data distribution:
- [ ] dropoff_site HORNSBY BEND, load type BRUSH
- [ ] dropoff_site ORGANICS BY GOSH, load type ORGANICS

These combinations are not empty NaN rows. So we will replace it with mean.

In [122]:
for dropoff_site in df4.loc[df4['load_weight'].isna(),'dropoff_site'].unique():
    for load_type in df4.loc[(df4['dropoff_site'] == dropoff_site) & (df4['load_weight'].isna()), 'load_type'].unique():
        dropoff_site_load_type_skew = getSkew(dropoff_site=dropoff_site, load_type=load_type)
        print(f"dropoff_site {dropoff_site} load_type {load_type} skew {dropoff_site_load_type_skew}")
        display(df4.loc[(df4['dropoff_site'] == dropoff_site) & (df4['load_type'] == load_type)])

df5 = df4.copy()

df5.loc[(df5['dropoff_site'] == 'HORNSBY BEND') 
        & (df5['load_type'] == 'BRUSH') 
        & (df5['load_weight'].isna()), 'load_weight'] = df5.loc[(df5['dropoff_site'] == 'HORNSBY BEND') 
                                                                & (df5['load_type'] == 'BRUSH'), 'load_weight'] \
                                                            .mean()

df5.loc[(df5['dropoff_site'] == 'ORGANICS BY GOSH') 
        & (df5['load_type'] == 'ORGANICS') 
        & (df5['load_weight'].isna()), 'load_weight'] = df5.loc[(df5['dropoff_site'] == 'HORNSBY BEND') 
                                                                & (df5['load_type'] == 'BRUSH'), 'load_weight'] \
                                                            .mean()

dropoff_site HORNSBY BEND load_type BRUSH skew 0.2636609436413962


,report_date,load_time,load_type,load_weight,dropoff_site
160,2021-03-20,2021-03-20 07:57:00 UTC,BRUSH,3800.0,HORNSBY BEND
161,2021-03-20,2021-03-20 08:23:00 UTC,BRUSH,4180.0,HORNSBY BEND
162,2021-03-20,2021-03-20 07:11:00 UTC,BRUSH,6500.0,HORNSBY BEND
163,2021-03-20,2021-03-20 10:41:00 UTC,BRUSH,5680.0,HORNSBY BEND
164,2021-03-20,2021-03-20 07:21:00 UTC,BRUSH,3180.0,HORNSBY BEND
...,...,...,...,...,...
26005,2021-03-01,2021-03-01 04:25:00 UTC,BRUSH,6020.0,HORNSBY BEND
26006,2021-03-01,2021-03-01 11:32:00 UTC,BRUSH,8180.0,HORNSBY BEND
26007,2021-03-01,2021-03-01 01:31:00 UTC,BRUSH,2940.0,HORNSBY BEND
26008,2021-03-01,2021-03-01 02:40:00 UTC,BRUSH,6680.0,HORNSBY BEND


dropoff_site ORGANICS BY GOSH load_type ORGANICS skew 0.07768898474656169


,report_date,load_time,load_type,load_weight,dropoff_site
775,2021-02-21,2021-02-21 10:41:00 UTC,ORGANICS,2480.0,ORGANICS BY GOSH
776,2021-03-14,2021-03-14 01:37:00 UTC,ORGANICS,21700.0,ORGANICS BY GOSH
777,2021-03-21,2021-03-21 07:56:00 UTC,ORGANICS,7120.0,ORGANICS BY GOSH
778,2021-02-21,2021-02-21 11:07:00 UTC,ORGANICS,5560.0,ORGANICS BY GOSH
779,2021-03-14,2021-03-14 03:04:00 UTC,ORGANICS,20460.0,ORGANICS BY GOSH
...,...,...,...,...,...
26151,2021-03-01,2021-03-01 11:19:00 UTC,ORGANICS,18580.0,ORGANICS BY GOSH
26152,2021-03-01,2021-03-01 11:26:00 UTC,ORGANICS,12100.0,ORGANICS BY GOSH
26153,2021-03-01,2021-03-01 04:27:00 UTC,ORGANICS,10900.0,ORGANICS BY GOSH
26154,2021-03-01,2021-03-01 05:55:00 UTC,ORGANICS,21800.0,ORGANICS BY GOSH


,report_date,load_time,load_type,load_weight,dropoff_site


## Data Exploration

## Conclusions, Assumptions, Overall Analysis

`put your conclusions here`

## Playground

### Problem 1

2. eksplorasi data
a. rentang waktu pengambilan data
b. tipe load sampah
```
df.loc[:,'load_type'].unique()
```
c. tempat pembuangan sampah
```
df.loc[:,'dropoff_site'].unique()
```

3. insight/infromasi untuk masing-masing site.
```
df.loc[df['dropoff_site'] == 'MRF'].groupby(['dropoff_site'])['load_weight'].agg([pd.Series.mean, pd.Series.median, pd.Series.mode])
```

4. site paling menarik + alasan

5. site tersebut: column `load_weight` ada outlier? berapa persen? bandingkan mean, median, mode setelah datanya dibersihkan.

6. site tersebut: range, variance, standar deviasi column `load_weight`

pg_df_MRF = df.loc[df['dropoff_site'] == 'MRF','load_weight']
pg_df_MRF.max() - pg_df_MRF.min()
pg_df_MRF.var()
pg_df_MRF.std() # np.sqrt(pandas.Series.var())

7. menambah site baru: berapa kapasitas penampungan di site baru berdasarkan perhitungan confidence interval.

8. uji hipotesis

1. teknik handling outlier

2. konsep dibaling confidence interval

3. uji hipotesis

### Problem 2

forecasting.

v = Δw / Δt 

In [ ]:
df.loc[:,'load_type'].unique()
df.loc[:,'dropoff_site'].unique()
# force pandas to suppress the scientific notation
# pd.options.display.float_format = '{:,.2f}'.format
df.loc[df['dropoff_site'] == 'MRF'].groupby(['dropoff_site'])['load_weight'].agg([pd.Series.mean, pd.Series.median, pd.Series.mode])